In [2]:
from Sentiment_analysis import *

from unidecode import unidecode

import pandas as pd
import numpy as np

import proj_base

from os import listdir

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [ ]:
# lst = []
# file_list = sorted(listdir('./Review_Texts/'))
# for i in range(500):
#     filename = file_list[i]
# #     intColumns = ['No. Reader', 'No. Helpful', 'Cleanliness','Check in / front desk', 'Value', 'Overall', 'Service', 'Business service', 'Rooms', 'Location']
#     characterThreshold = 60
#     with open('./Review_Texts/'+filename, 'r') as content_file:
#         content = content_file.read()
        
#         #print(repr(content))
#     if content.count("\r") > 0:
#         reviews = content.split("\r\n\r\n")
#     else:
#         reviews = content.split("\n\n")
    
#     count = 0
#     for r in reviews:
#         thisReview = {}
#         splt = r.split("\n")
#         for s in splt:
#             for c in ['Content']:
#                 if "<"+c+">" in s:
#                     value = s.replace('<'+c+'>', '')
#                     if c in intColumns:
#                         value = int(value)
#                     if value == -1: #we dont want -1 as this is going to mess up averaging, take np.nan
#                         value = np.nan

#                     if c == "Content":
#                         value = unidecode(unicode(value.lower(), encoding="utf-8"))

#                     thisReview[c] = value
                    
#         if not thisReview.get("Content") == None and len(thisReview["Content"]) > 60:
#             #only add if theres content and its long enough
#             count += 1
            
#     lst.append(count)

In [ ]:
# lst

# lst[:0]

# data = pd.read_csv('data.csv')

# data.ix[:lst[0],'Hotels'] = 1
# j = 2
# for i in range(1,len(lst)):
#     offset = sum(lst[:i])
#     data.ix[offset:offset+lst[i],'Hotels'] = j
#     j += 1

In [ ]:
# data.to_csv('data2.csv')

In [ ]:
data = proj_base.getStandardData(numFiles=50)
print(data.shape)
data.head()

In [ ]:
# keywords are calculated using the first 50 hotels, though entire data is 500 hotels. so first import 50 hotels, get keywords
# and then import data.csv

In [ ]:
proj_base.aspectSegmentationChiSquared(data)

In [ ]:
data = pd.read_csv('data2.csv')

In [ ]:
for aspect in proj_base.seeds:
    proj_base.aspect = aspect
    train = proj_base.getTrainingData(data)
    train.to_csv('train'+'_'.join(aspect.split('/'))+'.csv')
    print aspect

In [ ]:
del data

In [ ]:
df = pd.read_csv('trainCheck in _ front desk.csv')

In [ ]:
df = df['Review Number']

In [ ]:
# df2 = pd.read_csv('trainCleanliness.csv')
# df2 = pd.read_csv('trainLocation.csv')
# df2 = pd.read_csv('trainRooms.csv')
# df2 = pd.read_csv('trainService.csv')
# df2 = pd.read_csv('trainValue.csv')

In [ ]:
from pandasql import *
pysqldf = lambda q: sqldf(q, globals())

q = '''
select a.'Review Number' from df as a inner join df2 as b on a.'Review Number'
= b.'Review Number'
'''

In [ ]:
df = pysqldf(q)

df.shape

In [ ]:
df.to_csv('allAspectTrainingData.csv')

# Start

In [80]:
train = pd.read_csv('trainValue.csv')

In [81]:
import ast
train['aspectSentences2'] = train['aspectSentences'].copy()
train['aspectSentences'] = train['aspectSentences'].apply(ast.literal_eval)

In [82]:
train['aspectRating'].value_counts()

5.0    16292
4.0    12561
3.0     6450
2.0     3371
1.0     2747
Name: aspectRating, dtype: int64

In [112]:
train.ix[train['aspectRating'] == 2, 'aspectRating'] = 1
train.ix[train['aspectRating'] == 3, 'aspectRating'] = 2
train.ix[(train['aspectRating'] == 4) | (train['aspectRating'] == 5), 'aspectRating'] = 3

train['aspectRating'].value_counts()

3.0    28853
2.0     6450
1.0     6118
Name: aspectRating, dtype: int64

In [113]:
mins = train['aspectRating'].value_counts().min()
train_balanced = train.groupby("aspectRating").apply(lambda x: x.sample(n=mins))

In [114]:
train_balanced['aspectRating'].value_counts()

3.0    6118
2.0    6118
1.0    6118
Name: aspectRating, dtype: int64

In [115]:
train_balanced = train_balanced.apply(addSentimentScores, axis = 1)

In [116]:
X_senti = train_balanced.iloc[:,8:]

In [117]:
from sklearn import preprocessing

In [118]:
min_max_scaler = preprocessing.MinMaxScaler()
X_senti = min_max_scaler.fit_transform(X_senti)

In [120]:
import string
import sklearn.feature_extraction 
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
# from sklearn.naive_bayes import GaussianNBa

In [121]:
def custom_tokenizer(review):  
    review = review.encode('ascii',errors='ignore')
    review = review.translate(None,string.punctuation)
    words = nltk.tokenize.word_tokenize(review)
    words = [nltk.stem.porter.PorterStemmer().stem(word) for word in words]
    # stop words removed by vectorizer
    
    return words

In [122]:
from sklearn.decomposition import PCA

In [123]:
TfidVect = sklearn.feature_extraction.text.TfidfVectorizer(min_df=10, ngram_range=(1,2), tokenizer = custom_tokenizer,
                                                      stop_words = nltk.corpus.stopwords.words('english'))
X_tfidf = TfidVect.fit_transform(train_balanced['aspectSentences2'].tolist())
X_tfidf = X_tfidf.toarray()

In [124]:
newX = np.concatenate((X_tfidf,X_senti), axis = 1)

In [125]:
lin_clf = sklearn.svm.LinearSVC()

In [126]:
accuracy = 0
mse = 0
corr = 0
for i in range(5):

    X_train, X_test, y_train, y_test = train_test_split(newX, train_balanced['aspectRating'].tolist() )

    pca = PCA(n_components=100)
    pca.fit(X_train)
    X_train2 = pca.transform(X_train)
    X_test2 = pca.transform(X_test)

#     clf.fit(X_train2,y_train)
#     y_predicted = clf.predict(X_test2)
    
    lin_clf.fit(X_train2,y_train)
    y_predicted = lin_clf.predict(X_test2)

#     lin_clf.fit(X_train,y_train)
#     y_predicted = lin_clf.predict(X_test)
    
    accuracy += (pd.Series(y_predicted) == pd.Series(y_test)).mean()
    mse +=  ((pd.Series(y_predicted)-pd.Series(y_test))**2).mean()
    corr += np.corrcoef(y_predicted,y_test)

In [127]:
print 'accuracy {}'.format(accuracy/5.0)
print 'mse {}'.format(mse/5.0)
print 'corr {}'.format(corr/5.0)

accuracy 0.612856831554
mse 0.601568969274
corr [[ 1.          0.58016075]
 [ 0.58016075  1.        ]]


In [ ]:
# baseline
y_predicted = train_balanced['Overall'].tolist()
y_test = train_balanced['aspectRating'].tolist()
print 'accuracy'
print (pd.Series(y_predicted) == pd.Series(y_test)).mean()
print
print 'mse'
print ((pd.Series(y_predicted)-pd.Series(y_test))**2).mean()
print
print 'corr'
print np.corrcoef(y_predicted,y_test)

In [ ]:
clf = GaussianNB()

In [ ]:
lin_clf.fit(X_train2,y_train)
y_predicted = lin_clf.predict(X_test2)

print (pd.Series(y_predicted) == pd.Series(y_test)).mean()
print ((pd.Series(y_predicted)-pd.Series(y_test))**2).mean()

In [ ]:
def select_top(clf):
    top = []
    for i in range(5):
        top += list(np.argsort(clf.coef_[i])[-500:])
        
    return top

In [ ]:
top_indices = select_top(lin_clf)

In [ ]:
sorted(top_indices)[-6:]

In [ ]:
X_train2 = np.concatenate((X_train[:,0:5], X_train[:,top_indices]), axis=1)
X_test2 = np.concatenate((X_test[:,0:5], X_test[:,top_indices]), axis=1)

lin_clf.fit(X_train2,y_train)
y_predicted = lin_clf.predict(X_test2)

(pd.Series(y_predicted) == pd.Series(y_test)).mean()

In [ ]:
np.corrcoef(y_predicted,y_test)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, train_balanced['aspectRating'].tolist() )

In [ ]:
lin_clf.fit(X_train,y_train)
y_predicted = lin_clf.predict(X_test)

In [ ]:
print (pd.Series(y_predicted) == pd.Series(y_test)).mean()
print ((pd.Series(y_predicted)-pd.Series(y_test))**2).mean()

In [ ]:
TfidVect = sklearn.feature_extraction.text.TfidfVectorizer(min_df=10, ngram_range=(1,1), tokenizer = custom_tokenizer,
                                                      stop_words = nltk.corpus.stopwords.words('english'))
X_tfidf = TfidVect.fit_transform(train_balanced['aspectSentences'].tolist())
X_tfidf = X_tfidf.toarray()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, train_balanced['aspectRating'].tolist() )

In [ ]:
lin_clf = sklearn.svm.LinearSVC()
lin_clf.fit(X_train,y_train)
y_predicted = lin_clf.predict(X_test)

In [ ]:
(pd.Series(y_predicted) == pd.Series(y_test)).mean()


In [ ]:
from sklearn.naive_bayes import *

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(newX, train_balanced['aspectRating'].tolist() )

In [ ]:
clf = GaussianNB()
clf.fit(X_train,y_train)
y_predicted = clf.predict(X_test)
(pd.Series(y_predicted) == pd.Series(y_test)).mean()

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(lin_clf, X_tfidf, train_balanced['aspectRating'].tolist(), cv = 5)
scores.mean()

In [101]:
import matplotlib.pyplot as plt

In [109]:
from sklearn.metrics import confusion_matrix
import itertools

In [110]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i,"{0:.2f}".format(cm[i, j]),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [111]:
# Test confusion matrix function
y_pred = y_predicted
class_names=range(1,6)
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')


plt.show()

Confusion matrix, without normalization
[[501 115  31  13  28]
 [273 180 109  39  73]
 [109 100 199 124 157]
 [ 40  60 125 172 277]
 [ 56  39  58 140 416]]
Normalized confusion matrix
[[ 0.72819767  0.16715116  0.04505814  0.01889535  0.04069767]
 [ 0.40504451  0.26706231  0.16172107  0.0578635   0.10830861]
 [ 0.15820029  0.14513788  0.28882438  0.17997097  0.22786647]
 [ 0.05934718  0.08902077  0.18545994  0.25519288  0.41097923]
 [ 0.07898449  0.05500705  0.08180536  0.19746121  0.58674189]]
